In [1]:
import sys, pandas as pd, numpy as np
sys.path.append('..')
from src.task2_config import DATA_PATH
from src.loader import load_real_data
from src.chunker import chunk_all_complaints
from src.embedder import generate_all_embeddings
from src.vectorstore import create_vectorstore, search
from src.task2_reporter import create_final_summary
print("✅ Setup complete - Processing ALL your real data")

✅ Config loaded - Data path: d:\10 acadamy\Intelligent Complaint Analysis for Financial Services\notebooks\..\data\processed\filtered_complaints.csv


d:\10 acadamy\Intelligent Complaint Analysis for Financial Services\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Setup complete - Processing ALL your real data


In [2]:
# Cell 2: Load YOUR processed data from Task 1
from pathlib import Path

processed_file = Path('../data/processed/filtered_complaints.csv')
print(f"📂 Loading from: {processed_file}")
df = pd.read_csv(processed_file)
print(f"✅ Loaded {len(df):,} processed complaints")
print(f"📊 Products:\n{df['Product_Category'].value_counts()}")

📂 Loading from: ..\data\processed\filtered_complaints.csv
✅ Loaded 265,694 processed complaints
📊 Products:
Product_Category
Mortgage        130134
Credit Card      80620
Student Loan     53194
Payday Loan       1746
Name: count, dtype: int64


In [3]:
df = load_real_data()
print(f"\n📊 Processing ALL {len(df):,} complaints")
print(f"📋 Columns: {df.columns.tolist()}")

📂 Loading YOUR real data: d:\10 acadamy\Intelligent Complaint Analysis for Financial Services\notebooks\..\data\processed\filtered_complaints.csv
✅ Loaded 265,694 complaints
📋 Columns: ['Complaint ID', 'Date received', 'Product', 'Product_Category', 'Issue', 'Company', 'State', 'Consumer complaint narrative', 'Cleaned_Narrative']

📊 Processing ALL 265,694 complaints
📋 Columns: ['Complaint ID', 'Date received', 'Product', 'Product_Category', 'Issue', 'Company', 'State', 'Consumer complaint narrative', 'Cleaned_Narrative']


In [4]:
import sys, pandas as pd
sys.path.append('..')
from src.task2_config import DATA_PATH

# Load and inspect
df = pd.read_csv(DATA_PATH)
print(f"✅ Loaded {len(df):,} complaints")
print(f"\n📋 COLUMNS:")
for col in df.columns:
    print(f"  • '{col}'")
    
# Check the narrative column
narrative_col = 'Consumer complaint narrative'
if narrative_col in df.columns:
    print(f"\n📝 Sample narrative (first 100 chars):")
    print(df[narrative_col].iloc[0][:100] if pd.notna(df[narrative_col].iloc[0]) else "EMPTY")
    
    # Count non-empty narratives
    non_empty = df[narrative_col].notna().sum()
    print(f"\n📊 Complaints with narratives: {non_empty:,}/{len(df):,} ({(non_empty/len(df)*100):.1f}%)")

✅ Loaded 265,694 complaints

📋 COLUMNS:
  • 'Complaint ID'
  • 'Date received'
  • 'Product'
  • 'Product_Category'
  • 'Issue'
  • 'Company'
  • 'State'
  • 'Consumer complaint narrative'
  • 'Cleaned_Narrative'

📝 Sample narrative (first 100 chars):
I signed a purchase agreement with Lennar Corporation on XX/XX/year>, for a new construction home in

📊 Complaints with narratives: 265,694/265,694 (100.0%)


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm



splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Chunking"):
    text = str(row['Consumer complaint narrative'])
    if len(text) < 10: continue
    for i, chunk in enumerate(splitter.split_text(text)):
        chunks.append({
            'chunk_id': len(chunks),
            'complaint_id': idx,
            'product': row['Product_Category'],
            'chunk_index': i,
            'total_chunks': len(splitter.split_text(text)),
            'chunk_text': chunk,
            'chunk_length': len(chunk)
        })

chunks_df = pd.DataFrame(chunks)
chunks_df.to_parquet('../data/chunks/all_chunks.parquet')
print(f"✅ Created {len(chunks_df):,} chunks")

Chunking: 100%|██████████| 265694/265694 [53:18<00:00, 83.08it/s]  


✅ Created 1,051,455 chunks


In [6]:
from collections import Counter
import time
import os

class SimpleEmbedder:
    def __init__(self, vocab_size=500): 
        self.vocab_size = vocab_size
        self.word_to_idx = {}
    
    def fit(self, texts):
        word_counts = Counter()
        for text in texts[:10000]:
            word_counts.update(str(text).lower().split())
        for i, (w, _) in enumerate(word_counts.most_common(self.vocab_size)):
            self.word_to_idx[w] = i
        return self
    
    def encode(self, texts):
        embeddings = []
        for text in tqdm(texts, desc="Embedding"):
            vec = np.zeros(self.vocab_size)
            for w in str(text).lower().split():
                if w in self.word_to_idx:
                    vec[self.word_to_idx[w]] += 1
            norm = np.linalg.norm(vec)
            embeddings.append(vec/norm if norm > 0 else vec)
        return np.array(embeddings)

# Now chunks_df exists from Cell 3
texts = chunks_df['chunk_text'].tolist()
embedder = SimpleEmbedder(500).fit(texts)

# Save in batches
os.makedirs('simple_embeddings', exist_ok=True)
start = time.time()
for i in range(0, len(texts), 10000):
    batch = texts[i:i+10000]
    np.save(f'simple_embeddings/batch_{i//10000:04d}.npy', embedder.encode(batch))
    print(f"Batch {i//10000+1}: {min(i+10000, len(texts)):,}/{len(texts):,} - {(time.time()-start)/60:.1f} min")

print(f"\n✅ Done: {len(texts):,} embeddings in {(time.time()-start)/60:.1f} min")

Embedding: 100%|██████████| 10000/10000 [00:02<00:00, 3352.53it/s]


Batch 1: 10,000/1,051,455 - 0.1 min


Embedding: 100%|██████████| 10000/10000 [00:02<00:00, 3825.04it/s]


Batch 2: 20,000/1,051,455 - 0.1 min


Embedding: 100%|██████████| 10000/10000 [00:02<00:00, 3710.52it/s]


...

Batch 104: 1,040,000/1,051,455 - 4.4 min


Embedding: 100%|██████████| 10000/10000 [00:03<00:00, 2999.99it/s]


Batch 105: 1,050,000/1,051,455 - 4.4 min


Embedding: 100%|██████████| 1455/1455 [00:00<00:00, 5577.50it/s]


Batch 106: 1,051,455/1,051,455 - 4.5 min

✅ Done: 1,051,455 embeddings in 4.5 min


In [ ]:
import faiss, pickle, numpy as np, pandas as pd
from pathlib import Path

e = np.vstack([np.load(f).astype('float32') for f in sorted(Path('simple_embeddings').glob('batch_*.npy'))])
faiss.normalize_L2(e); i = faiss.IndexFlatIP(e.shape[1]); i.add(e)
os.makedirs('vector_store', exist_ok=True); faiss.write_index(i, 'vector_store/faiss_index.idx')
c = pd.read_parquet('../data/chunks/all_chunks.parquet')
pickle.dump([{'chunk_id':int(r.chunk_id),'complaint_id':int(r.complaint_id),'product':r.product,'chunk_text':r.chunk_text[:200]} for _,r in c.iterrows()], open('vector_store/metadata.pkl','wb'))
print(f"✅ FAISS: {i.ntotal:,} | Metadata: {len(c):,}")

In [ ]:
def search(query, k=3):
    # Simple query embedding
    q_vec = np.zeros(500)
    for w in query.lower().split():
        if w in embedder.word_to_idx:
            q_vec[embedder.word_to_idx[w]] += 1
    q_vec = q_vec / np.linalg.norm(q_vec) if np.linalg.norm(q_vec) > 0 else q_vec
    
    # Search
    scores, indices = index.search(q_vec.reshape(1, -1).astype('float32'), k)
    
    print(f"\n🔍 Query: '{query}'")
    for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
        if idx < len(metadata):
            print(f"\n  {i+1}. Score: {score:.3f} | Product: {metadata[idx]['product']}")
            print(f"     {metadata[idx]['chunk_text'][:150]}...")

# Test
search("credit card fraud")
search("mortgage payment late")